In [34]:
import sys
sys.path.insert(1, '/scratch/cinthiasouza/mv-text-summarizer')

import glob, os
import pandas as pd
import json
import spacy
import nltk
import numpy as np
import json
#import smogn
import seaborn as sns
import pickle


from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,classification_report
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from src import utils_classification as utils
from sklearn.metrics import matthews_corrcoef
from sklearn.covariance import EllipticEnvelope

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

from multiprocessing import Process, Queue

In [3]:
with open('dataset2.p', 'rb') as fp:
    dataset = pickle.load(fp)

In [14]:
def oneclass_svm(X_train):
    
    svm = OneClassSVM(gamma='scale', kernel='rbf', nu=0.67)
    svm.fit(X_train)
    
    return svm

In [16]:
def oneclass_rf(X_train):
    
    rf = IsolationForest(n_estimators=1000, contamination=0.67,warm_start=True, n_jobs=-1)
    rf.fit(X_train)
    
    return rf

In [38]:
def train(dataset, name_models, sections, train_class=1):
    
    models = {}
    for section in sections:
        
        X_train = dataset[section][0].copy()
        y_train = dataset[section][2].copy()
        
        if train_class != 'all':
            X_train = X_train[y_train==train_class]
        else:
            X_train = X_train
        
        aux = {}
        for name_model in name_models:
        
            if name_model == "svm":
                aux['svm'] = oneclass_svm(X_train)
            
            if name_model == "rf":
                aux['rf'] = oneclass_rf(X_train)
                
        models[section] = aux
        
    return models
        
def predict(dataset, models, name_models, sections):

    
    results= {}
    predictions = {}
    cm = {}
    
    for section in sections:
        
        X_test = dataset[section][1].copy()
        y_test = dataset[section][3].copy()

        y_test[y_test == 1] = 1
        y_test[y_test == 0] = -1
        
        aux_results = {}
        aux_predictions = {}
        aux_cm = {}
        
        for name_model in name_models:
            
            aux_predictions[name_model] = models[section][name_model].predict(X_test)
            
            aux_results[name_model] = classification_report(y_test, aux_predictions[name_model],  labels=[-1, 1])
            
            aux_cm[name_model] = [confusion_matrix(y_test, aux_predictions[name_model]).ravel()]
    
        predictions[section] = aux_predictions
        results[section] = aux_results
        cm[section] = aux_cm
    
    return predictions, results, cm

In [30]:
models = train(dataset, name_models, sections, train_class=1) 

In [39]:
predictions, results, cm = predict(dataset, models, name_models, sections)

In [41]:
results

{'introduction': {'rf': '              precision    recall  f1-score   support\n\n          -1       0.82      0.58      0.68      4482\n           1       0.13      0.34      0.19       846\n\n    accuracy                           0.54      5328\n   macro avg       0.48      0.46      0.43      5328\nweighted avg       0.71      0.54      0.60      5328\n',
  'svm': '              precision    recall  f1-score   support\n\n          -1       0.82      0.59      0.68      4482\n           1       0.13      0.33      0.19       846\n\n    accuracy                           0.54      5328\n   macro avg       0.48      0.46      0.44      5328\nweighted avg       0.71      0.54      0.60      5328\n'},
 'materials': {'rf': '              precision    recall  f1-score   support\n\n          -1       0.87      0.60      0.71      5574\n           1       0.11      0.36      0.17       758\n\n    accuracy                           0.57      6332\n   macro avg       0.49      0.48      0.44 

In [40]:
tn, fp, fn, tp

{'introduction': {'rf': [array([2589, 1893,  557,  289])],
  'svm': [array([2622, 1860,  567,  279])]},
 'materials': {'rf': [array([3324, 2250,  485,  273])],
  'svm': [array([3531, 2043,  478,  280])]},
 'concat': {'rf': [array([13150,  7657,  1578,   856])],
  'svm': [array([13263,  7544,  1539,   895])]}}

In [31]:
models

{'introduction': {'rf': IsolationForest(contamination=0.67, n_estimators=1000, n_jobs=-1,
                  warm_start=True),
  'svm': OneClassSVM(nu=0.67)},
 'materials': {'rf': IsolationForest(contamination=0.67, n_estimators=1000, n_jobs=-1,
                  warm_start=True),
  'svm': OneClassSVM(nu=0.67)},
 'concat': {'rf': IsolationForest(contamination=0.67, n_estimators=1000, n_jobs=-1,
                  warm_start=True),
  'svm': OneClassSVM(nu=0.67)}}

In [26]:
name_models = ['rf', 'svm']
sections = ['introduction', 'materials', 'concat']

l2 = Queue()
p2 = Process(
    target=predict, args=(
         dataset, models, name_models, sections))
p2.start()   

Process Process-5:
Traceback (most recent call last):
  File "/scratch/cinthiasouza/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/scratch/cinthiasouza/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_7930/754197088.py", line 48, in predict
    aux_predictions[name_model] = models[section][name_model].predict(X_test)
TypeError: 'Process' object is not subscriptable


In [ ]:
l1 = Queue()
p1 = Process(
    target=train_test, args=(
         svm, X_train, X_test, y_train,y_test, 1))
p1.start()   

In [ ]:
rf = IsolationForest(n_estimators=1000, contamination=0.67warm_start=True, n_jobs=-1)
y_pred, y_test, score, tn, fp, fn, tp = train_test(rf, X_train, X_test, y_train,y_test, train_class=1)
print("True Negative: {} - False Positive: {} - False Negative: {} - True Positive: {}".format(tn, fp, fn, tp))

In [ ]:
ee = EllipticEnvelope(contamination=0.67)
y_pred, y_test, score, tn, fp, fn, tp = train_test(ee, X_train, X_test, y_train,y_test, train_class=1)
print("True Negative: {} - False Positive: {} - False Negative: {} - True Positive: {}".format(tn, fp, fn, tp))


In [ ]:
trainX = X_train

nu = (sum(y_test))/trainX.shape[0]

svm = OneClassSVM(gamma='scale', kernel='rbf', nu=0.67)
rf = IsolationForest(n_estimators=1000, contamination=0.67warm_start=True, n_jobs=-1)
ee = EllipticEnvelope(contamination=0.67)

#svm.fit(trainX)
rf.fit (trainX)
#ee.fit (trainX)

pred_svm = svm.predict(X_test)
pred_rf = rf.predict(X_test)
pred_ee = ee.predict(X_test)

y_test[y_test == 1] = 1
y_test[y_test == 0] = -1

#score = classification_report(y_test, pred_svm, labels=[1, -1])
#print( score)

score = classification_report(y_test, pred_rf,  labels=[1, -1])
print(score)

#score = classification_report(y_test, pred_ee,  labels=[1, -1])
#print(score)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred_svm).ravel()
print("True Negative: {} - False Positive: {} - False Negative: {} - True Positive: {}".format(tn, fp, fn, tp))

tn, fp, fn, tp = confusion_matrix(y_test, pred_rf).ravel()
print("\nTrue Negative: {} - False Positive: {} - False Negative: {} - True Positive: {}".format(tn, fp, fn, tp))

tn, fp, fn, tp = confusion_matrix(y_test, pred_ee).ravel()
print("\nTrue Negative: {} - False Positive: {} - False Negative: {} - True Positive: {}".format(tn, fp, fn, tp))